In [4]:
import sys
print(sys.executable)
print(sys.path)


C:\Users\vishn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\python311.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\vishn\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages', 'C:\\Users\\vishn\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\win32', 'C:\\Users\\vishn\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python31

In [5]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



In [6]:
folder_path=r"C:\Users\vishn\OneDrive\Desktop\SUBASH_BTP\archive\dataset\train"

#Define image transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize (adjust if needed)
])

# Load dataset using ImageFolder
dataset = datasets.ImageFolder(root=folder_path, transform=transform)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
# Check a sample batch
data_iter = iter(dataloader)
images, labels = next(data_iter)

print(f"Loaded batch shape: {images.shape}")
print(f"Labels: {labels}")

Loaded batch shape: torch.Size([32, 1, 224, 224])
Labels: tensor([0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,
        1, 0, 0, 0, 1, 1, 0, 1])


In [8]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import cv2  # For image processing

In [9]:
# Define dataset path
dataset_path = r"C:\Users\vishn\OneDrive\Desktop\SUBASH_BTP\archive\dataset\train"

# Define classes and labels
classes = {"fractured": 1, "non_fractured": 0}
image_size = (224, 224)  # Resize images to this size

# Load images and labels
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        if img is not None:
            img = cv2.resize(img, image_size)  # Resize
            images.append(img)
            labels.append(label)
    return images, labels

# Load fractured images
fractured_images, fractured_labels = load_images_from_folder(os.path.join(dataset_path, "fractured"), classes["fractured"])

# Load non-fractured images
non_fractured_images, non_fractured_labels = load_images_from_folder(os.path.join(dataset_path, "not fractured"), classes["non_fractured"])

# Combine all images and labels
all_images = np.array(fractured_images + non_fractured_images, dtype=np.float32)
all_labels = np.array(fractured_labels + non_fractured_labels, dtype=np.int64)

# Normalize pixel values to [0, 1]
all_images /= 255.0

# Convert to PyTorch tensors
all_images = torch.tensor(all_images).unsqueeze(1)  # Add channel dimension
all_labels = torch.tensor(all_labels)

In [10]:
all_images[0].shape

torch.Size([1, 224, 224])

In [ ]:
# Create PyTorch Dataset
class BoneFractureDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Create Dataset and DataLoader
dataset = BoneFractureDataset(all_images, all_labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

# Check DataLoader
data_iter = iter(dataloader)
sample_images, sample_labels = next(data_iter)
print(f"Batch shape: {sample_images.shape}")
print(f"Sample labels: {sample_labels}")

Batch shape: torch.Size([32, 1, 224, 224])
Sample labels: tensor([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 1, 0, 1, 1])


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

# Define CNN Model
class BoneFractureCNN(nn.Module):
    def __init__(self):
        super(BoneFractureCNN, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(128, 1),  
            nn.Sigmoid()  
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x

# Create model instance
model = BoneFractureCNN()
print(model)


BoneFractureCNN(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=100352, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3):

In [13]:
# Hyperparameters
learning_rate = 0.001
batch_size = 16
epochs = 10

# Loss function & optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Dataloader
train_dataset = data.TensorDataset(all_images, all_labels.float().unsqueeze(1))
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/10], Loss: 2.2512
Epoch [2/10], Loss: 0.3992
Epoch [3/10], Loss: 0.2698
Epoch [4/10], Loss: 0.1748
Epoch [5/10], Loss: 0.1126
Epoch [6/10], Loss: 0.1639
Epoch [7/10], Loss: 0.1474
Epoch [8/10], Loss: 0.0856
Epoch [9/10], Loss: 0.1486
Epoch [10/10], Loss: 0.1078
